In [1]:
from availableTools import *
from langfuse import Langfuse
from langfuse.openai import OpenAI
from langfuse.decorators import observe
import json
from jinjaProcessor import process_template
import requests

In [2]:
langfuse = Langfuse()

# llm_api_keys

In [3]:
api_keys = []
with open("llm_api_keys.txt") as f:
    for line in f.readlines():
        api_keys.append(line.strip())

current_index = 0

# Client config

In [4]:
def get_client(index):
    client = OpenAI(
        base_url="https://api.groq.com/openai/v1",
        api_key= api_keys[index]
    )
    return client

# Function to generate response

In [14]:
@observe()
def generate_response(messages,current_index):
    try:
        client = get_client(current_index)
        response = client.chat.completions.create(
            model = "llama-3.3-70b-versatile",
            messages = messages,
            temperature=0.1,
            top_p=0.1,
            presence_penalty=0.0,
            frequency_penalty=0.0,
        )
    except requests.exceptions.HTTPError as e:
        if response.status_code == 429:
            current_index = (current_index + 1) % len(api_keys)
            client = get_client(current_index)
            response = client.chat.completions.create(
                model = "llama-3.3-70b-versatile",
                messages = messages,
                temperature=0.1,
                top_p=0.1,
                presence_penalty=0.0,
                frequency_penalty=0.0,
            )
    return response

# Available tools

In [15]:
function_to_calls ={
    "getProductList": getProductList,
    "getCurrentCart": getCurrentCart,
    "addProduct": addProduct,
    "modifyCart": modifyCart,
    "removeProduct": removeProduct,
}

Function to end

In [16]:
def end_response(tool_call):
    end_response_func = ["NONE","greetings","out_of_scope","clarify_product_quantity"]
    if tool_call['function_name'] in end_response_func:
        return True
    else:
        return False

# System prompt

In [17]:
system_prompt = """
You are a Super Customer Service agent that also have tool calls capabilities.
You are not allowed to make up information or use any external sources other than your tools.
You should ALWAYS use the SAME language that the user use in your response. 
You can only response using the tools provided to you.

Here is the list of tools you can call:

{tools}

Reminder:
- Re-format all of your response and tool call response within <Response and tool call response> and <End of response and tool call response>
- Re-format function to calls within <Function call> and <End of function call>
- If there is no function to call, output "NONE" within <Function call> and <End of function call>.
- If there is no arguments to pass, output empty dictionary within "args" in <Function call>.
- ALWAYS output <Response and tool call response> first before your responses
- ALWAYS output one <Response and tool call response>, one <End of response and tool call response>, one <Function call> and one <End of function call> in your response.
- Do not mention function name outside of the <Function call> and <End of function call>.

Expected Response format:

<Response and tool call response>

Output your normal response here.

<End of response and tool call response>

<Function call>
[
    {{
        "function_name": "function_name here without backslash",
        "args": "indent function arguments here without backslash",
        "reason": "reason"
    }}
]
<End of function call>
"""

In [18]:
tools_str = process_template("tools_tempate.jinja")

In [19]:
system_prompt = system_prompt.format(tools=tools_str)
print(system_prompt)


You are a Super Customer Service agent that also have tool calls capabilities.
You are not allowed to make up information or use any external sources other than your tools.
You should ALWAYS use the SAME language that the user use in your response. 
You can only response using the tools provided to you.

Here is the list of tools you can call:

[
    {
        "type": "function",
        "function": {
            "name": "getProductList",
            "description": "You should always call this function whenever you want to see the available products in the store.",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "getCurrentCart",
            "description": "You should call this function whenever you want to see current product items in the cart",
            "parameters": {
                "type": "object",
                "propertie

# Conversation

In [20]:
def run_conversation(system_prompt,messages,new_chat_flag):
    if new_chat_flag:
        user_message = input("User: ")
        messages = [{
            "role": "system",
            "content": system_prompt
            }, {
            "role": "user",
            "content": user_message
        }]
        print("User: ", user_message)
    while True:
        response = generate_response(messages,current_index)
        response = response.choices[0].message
        print("Assistant: ", response.content)
        messages.append({
            "role": "assistant",
            "content": response.content
        })
        tool_calls = function_extractor(response)
        if (len(tool_calls) == 1 and end_response(tool_calls[0]) == True):
            break
    
        for tool_call in tool_calls:
            function_call = function_to_calls[tool_call['function_name']]
            function_args = tool_call['args']
            function_output = function_call(**function_args)
            content = json.dumps(function_output, indent=4)
            messages.append({
                "role": "tool",
                "tool_call_id": function_output['tool_call_id'],
                "content": content
            })    
    return messages

Chatbot

In [21]:
def run_chatbot():
    new_chat_flag = True
    messages = []
    while True:
        messages = run_conversation(system_prompt, messages, new_chat_flag)
        new_chat_flag = False
        user_message = input("User: ")
        print("User: ", user_message)
        if user_message.lower() == "exit":
            break
        else:
            messages.append({
                "role": "user",
                "content": user_message
            })

In [22]:
run_chatbot()

User:  woi mo beli indomie goreng ama indomie jumbo
Assistant:  <Response and tool call response>
Maaf, saya tidak bisa membantu Anda dengan permintaan tersebut karena tidak ada informasi yang cukup tentang produk Indomie Goreng dan Indomie Jumbo. Namun, saya bisa membantu Anda melihat daftar produk yang tersedia di toko atau membantu Anda dengan keranjang belanja Anda.
<End of response and tool call response>

<Function call>
[
    {
        "function_name": "getProductList",
        "args": {},
        "reason": "Untuk melihat daftar produk yang tersedia di toko"
    }
]
<End of function call>
Assistant:  <Response and tool call response>
Berikut adalah daftar produk Indomie yang tersedia di toko: Indomie Goreng, Indomie Goreng Jumbo, Indomie Goreng Jumbo Rasa Ayam Panggang, Indomie Kuah Rasa Kaldu Ayam, Indomie Kuah Rasa Ayam Bawang, Indomie Goreng Premium Japanese Takoyaki, Indomie Goreng Premium Japanese Shoyu, Indomie Goreng Premium Japanese Miso. Silakan pilih produk yang Anda i